##### Importing all the required packages

Host milvus in your local or feel free to modify the code using chromadb

In [1]:
import os
import textwrap
from langchain.chains import RetrievalQA
from langchain.vectorstores import Milvus
from langchain.document_loaders import WebBaseLoader

# Packages Lang Chain
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings

# Packages wrt Transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
from langchain.text_splitter import CharacterTextSplitter
from pymilvus import (connections, utility, FieldSchema, CollectionSchema, DataType, Collection,)



2023-07-20 15:36:18.862282: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Updating environment variables

In [2]:
update_env_variables = {"MILVUS_HOST": "localhost",
                        "MILVUS_PORT" : "19530"}
for key,val in update_env_variables.items():
    os.environ[key] = val

#### Check if milvus is up and running

In [3]:
connections.connect("default", host=update_env_variables["MILVUS_HOST"], port=update_env_variables["MILVUS_PORT"])

In [4]:
utility.list_collections()

['test_collection_2',
 'test_collection_1',
 'test_collection_3',
 'test_collection_4',
 'LangChainCollection',
 'test_collection_5',
 'test_collection_6',
 'test_collection']

#### Drop the collection which you are experimenting on

In [5]:
utility.drop_collection("test_collection_6")

Updated models as per the requirement:
<ul>
<li><a href="https://huggingface.co/google/flan-t5-base">flan-t5-base</a></li>
<li><a href="https://huggingface.co/google/flan-t5-large">flan-t5-large</a></li>
<li><a href="https://huggingface.co/google/flan-t5-xl">flan-t5-xl</a></li>
<li><a href="https://huggingface.co/decapoda-research/llama-7b-hf">llama-7b-hf</li>
<li><a href="https://huggingface.co/models?pipeline_tag=text2text-generation&sort=trending">more models</a></li>
<li></li>
</ul>

In [6]:
model_id = 'google/flan-t5-base'


Loading the models for llm

In [5]:
# tokenizer = T5Tokenizer.from_pretrained(model_id)
# model = T5ForConditionalGeneration.from_pretrained(model_id)

In [7]:
def load_llm(model_id, model=None, tokenizer=None, max_length=512):
    """
    loading model and tokenizer
    """
    try:
        if model is None:
            # tokenizer = AutoTokenizer.from_pretrained(model_id)
            # model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
            tokenizer = T5Tokenizer.from_pretrained(model_id, max_length=max_length, truncation=True)
            model = T5ForConditionalGeneration.from_pretrained(model_id)

        pipe = pipeline(
            "text2text-generation",
            model=model,
            tokenizer=tokenizer,
            max_length=max_length
        )
        local_llm = HuggingFacePipeline(pipeline=pipe)
        return local_llm
    except Exception as ex:
        print(f"Error while loading llm: {ex}")
    return None


##### Please modify max_length as per the input and requirements

In [8]:
local_llm = load_llm(model_id)
# local_llm = load_llm(model_id, max_length=1024)
local_llm

HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, pipeline=<transformers.pipelines.text2text_generation.Text2TextGenerationPipeline object at 0x15ba506a0>, model_id='gpt2', model_kwargs=None)

##### Downloading embeddings wrt model
Wrt flan-t5-base it's around 4gb

In [9]:
embeddings = HuggingFaceEmbeddings(model_name=model_id)
embeddings

No sentence-transformers model found with name /Users/fission/.cache/torch/sentence_transformers/google_flan-t5-base. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /Users/fission/.cache/torch/sentence_transformers/google_flan-t5-base were not used when initializing T5EncoderModel: ['lm_head.weight']
- This IS expected if you are initializing T5EncoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5EncoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': None, 'do_lower_case': False}) with Transformer model: T5EncoderModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
), model_name='google/flan-t5-base', cache_folder=None, model_kwargs={}, encode_kwargs={})

##### Loading few web urls and checking if question answering in working query LLM embeddings
<ul>
<li>Please feel free to go through the website and ask queries</li>
<li>Update urls and ask queries and validate results</li>
<li>Working well wrt single url wrt below parameters</li>
</ul>


In [10]:
loader = WebBaseLoader(
    [
        "https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm",
        "https://news.temple.edu/news/2023-07-14/who-was-oppenheimer-what-know-about-christopher-nolan-s-new-film",
     ]
)

##### Loading documents

In [11]:
documents = loader.load()
len(documents)

2

##### Checking the number of document and number of keys

In [12]:
document = documents[0]
document.__dict__.keys()

dict_keys(['page_content', 'metadata'])

In [13]:
document.metadata

{'source': 'https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm',
 'title': "Twitter's Recommendation Algorithm",
 'description': 'Twitter aims to deliver you the best of what’s happening in the world right now. This blog is an introduction to how the algorithm selects Tweets for your timeline.',
 'language': 'en_us'}

##### For the fist document viewing first 200 characters

In [14]:
document.page_content[:200]


"\n\n\n\n\nTwitter's Recommendation Algorithm\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEngineering\n\n\n\n\n\nBack\n\n\n\n\n\nEngineering\n\n\n\nInsights\n\n\n\n\nInfrastructure\n\n\n\n\nOpen source\n\n\n\n\n\n\nSign Up\n\n\n\n\n\n\n\n\n\nSign Up\n\n\n\n"

#### Please feel free fine-tune
<ul>
    <li>chunk_size</li>
    <li>chunk_overlap</li>
</ul>

In [15]:
text_splitter = CharacterTextSplitter(
    separator = " ",
    chunk_size = 512,
    chunk_overlap  = 32
) 


In [16]:
docs = []
for document in documents:
    # cleaning text
    document.page_content = document.page_content.encode("utf8","strict").decode("utf-8").encode("ascii", "ignore").decode("utf-8","strict").strip()

    # for few websites language is None
    document.metadata["language"] = "en_us"

    # using text splitter it's splitting documents into multiple documents
    output = text_splitter.create_documents([document.page_content], [document.metadata])

    # if pagecontent or metadata is None milvus won't take input parameters
    for o in output:
        if o is not None or o.page_content is not None:
            docs.append(o)


In [17]:
len(docs)

48

In [18]:
docs[0]

Document(page_content="Twitter's Recommendation Algorithm\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEngineering\n\n\n\n\n\nBack\n\n\n\n\n\nEngineering\n\n\n\nInsights\n\n\n\n\nInfrastructure\n\n\n\n\nOpen source\n\n\n\n\n\n\nSign Up\n\n\n\n\n\n\n\n\n\nSign Up\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nOpen source\nTwitter's Recommendation Algorithm\n\n\n\n            By\n            \n  \n  \n\n\n\n\nFriday, 31 March 2023\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLink copied successfully\n\n\n\n\n\n\n\n\n\nTwitter aims to deliver you the best of whats happening in the world right now. This requires a recommendation algorithm to distill the roughly", metadata={'source': 'https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm', 'title': "Twitter's Recommendation Algorithm", 'description': 'Twitter aims to deliver you the best of what’s happening in the world right now. This blog is an introduction to how the algorithm selec

#### Loading tokenized data and vectors into milvus database

In [19]:
vector_store = Milvus.from_documents(
    docs,
    embedding=embeddings,
    collection_name="test_collection_6",
    connection_args={"host": "localhost", "port": update_env_variables["MILVUS_PORT"]}
)

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

#### Testing on a template
Please feel free to update as per your use case and requirement

In [20]:
template_exp = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""



In [21]:
prompt_exp = PromptTemplate(
            input_variables=["context","question"],
            template=template_exp)

In [22]:
chain_type_kwargs = {"prompt": prompt_exp}
chain = RetrievalQA.from_chain_type(
    llm=local_llm,
    chain_type="stuff",

    # accessing the milvus vector database
    retriever=vector_store.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,

    # verbose=True
)


In [23]:
def chain_output(query, chain=chain, width=100):
    """
    running the query and printing the output response
    :param query:
    :param chain:
    :param width:
    :return:
    """
    try:
        response = chain.run(query)
        print("\n".join(textwrap.wrap(response, width=width)))
    except Exception as ex:
        print(f"Error while running the query: {ex}")

<a href="https://blog.twitter.com/engineering/en_us/topics/open-source/2023/twitter-recommendation-algorithm">twitter url</a>

In [24]:
%%time
query = "Explain the Twitter recommendation algorithm in 5 sentences"
chain_output(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The algorithm to distill the roughly 500 million Tweets posted daily down to a handful of top Tweets
that ultimately show up on your devices For You timeline. This blog is an introduction to how the
algorithm selects Tweets for your timeline. Our recommendation system is composed of many
interconnected services and jobs, which we will detail in this post. While there are many areas of
the app where Tweets are recommendedSearch, Explore, Adsthis post will focus on the home timelines
For Youfeed. How do we choose the For You timeline is called Home Mixer. Home Mixer is built on
Product Mixer, our custom Scala framework that facilitates building feeds of content. This service
acts as the software backbone that connects different candidate sources, scoring functions,
heuristics, and filters.
CPU times: user 47.1 s, sys: 3.38 s, total: 50.5 s
Wall time: 9.88 s


In [25]:
%%time
query1 = "What is Visibility Filtering?"
chain_output(query1)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

filter out Tweets based on their content and your preferences
CPU times: user 7.58 s, sys: 490 ms, total: 8.07 s
Wall time: 1.6 s


In [26]:
%%time
query2 = "Example of Visibility Filtering?"
chain_output(query2)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

remove Tweets from accounts you block or mute
CPU times: user 8.02 s, sys: 527 ms, total: 8.55 s
Wall time: 1.51 s


#### working well on second url query wrt second link: <a href="https://news.temple.edu/news/2023-07-14/who-was-oppenheimer-what-know-about-christopher-nolan-s-new-film">url</a>

In [27]:
%%time
query3 = "What is the plot of Oppenheimer film?"
chain_output(query3)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (532 > 512). Running this sequence through the model will result in indexing errors


The nuclear weapons Oppenheimer developed through the Manhattan Project famously helped bring to
know about Christopher Nolan's latest film Oppenheimer
CPU times: user 13.7 s, sys: 861 ms, total: 14.6 s
Wall time: 2.6 s


In [28]:
%%time
query4 = "Who is the director Oppenheimer film?"
chain_output(query4)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Christopher Nolan
CPU times: user 6.83 s, sys: 530 ms, total: 7.36 s
Wall time: 1.32 s
